# Урок 11: Mixture-of-Experts (MoE) Compression

MoE — это не только способ масштабирования моделей до триллионов параметров, но и мощный инструмент сжатия (через динамическую разреженность).

## 1. Математическая теория

### 1.1. Conditional Computation (Условные вычисления)
В MoE слое мы заменяем одну большую полносвязную сеть (MLP) на набор из $N$ маленьких «экспертов». Для каждого входного токена специальный слой-маршрутизатор (Router/Gate) выбирает только $k$ лучших экспертов:
$$y = \sum_{i=1}^k G(x)_i E_i(x)$$
Это позволяет модели иметь 100 млрд параметров (для хранения знаний), но использовать только 10 млрд для обработки каждого токена, что в 10 раз снижает FLOPs.

### 1.2. Методы сжатия из обзора:
*   **MoE-ification (Zhang et al., 2022):** Процесс превращения уже обученной плотной (dense) модели в MoE. Мы берем MLP слой и делим его нейроны на кластеры («экспертов»). 
*   **Sparse Upcycling (Komatsuzaki et al., 2023):** Метод инициализации MoE модели весами плотной модели с последующим дообучением. Это позволяет Hugging Face и NVIDIA выпускать MoE-версии своих моделей гораздо дешевле.
*   **Expert Pruning:** Удаление избыточных экспертов, которые редко выбираются маршрутизатором.
*   **QMoE (Frantar et al., 2024):** Квантование MoE моделей. Из-за огромного количества параметров MoE модели (например, Mixtral 8x7B) требуют экстремального сжатия до 2-3 бит для запуска на потребительском железе.

---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Expert(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
        )
    def forward(self, x):
        return self.net(x)

class MoELayer(nn.Module):
    def __init__(self, n_embd, num_experts=8, top_k=2):
        super().__init__()
        self.experts = nn.ModuleList([Expert(n_embd) for _ in range(num_experts)])
        self.gate = nn.Linear(n_embd, num_experts)
        self.top_k = top_k

    def forward(self, x):
        # x: (B, T, C)
        logits = self.gate(x) # (B, T, num_experts)
        weights, indices = torch.topk(logits, self.top_k, dim=-1)
        weights = F.softmax(weights, dim=-1)
        
        out = torch.zeros_like(x)
        for i, expert in enumerate(self.experts):
            # Маска: где этот эксперт был выбран
            mask = (indices == i).any(dim=-1)
            if mask.any():
                # Суммируем вклад эксперта с учетом веса из гейта
                batch_idx, token_idx = torch.where(mask)
                # Для простоты реализации представим логику взвешивания:
                # На практике используются более эффективные scatter/gather операции
                out[mask] += expert(x[mask]) # Здесь пропущено умножение на weight для краткости
        
        return out

print("nanoGPT Track: Реализован MoE слой с маршрутизацией Top-k.")

## 2. Промышленная реализация: Mixtral & Megablocks
В индустрии MoE используется для создания моделей с «дешевым инференсом». Например, **Mixtral 8x7B** имеет 47B параметров, но использует только 13B для каждого токена. Для эффективного обучения MoE используют библиотеку **Megablocks**, которая реализует «блочную разреженность» (Block-Sparse) для обхода проблем с производительностью стандартных слоев.

In [ ]:
from transformers import AutoModelForCausalLM
try:
    # Пример загрузки Mixtral — самой известной MoE модели
    model_id = "mistralai/Mixtral-8x7B-v0.1"
    # В индустрии MoE + Quantization — это стандарт доставки
    # model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True)
    print("Llama/Mistral Track: MoE позволяет запускать гигантские модели с ценой маленьких.")
except Exception as e:
    print(f"Ошибка: {e}")